In [2]:
import pandas as pd
import numpy as np
import feedparser
from bs4 import BeautifulSoup
import requests
import PyPDF2 as pdf
import io
import os
import re
import scrape_webpage
import scrape_pdf
import weblist

## Step 1: Scrape the list of webpages

In [5]:
new_orleans_url = "https://www.mvn.usace.army.mil/Missions/Regulatory/Public-Notices/"
web_list = weblist.get_web_list(new_orleans_url, "new_orleans")

# Save the weblist to csv to avoid repetitive running
# web_list.to_csv("new_orleans_weblist.csv")

,web_link,web_title,published_date,web_expire_date,pdf_url
0,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2015-01960-CM,9/4/2023,9/25/2023,https://www.sam.usace.army.mil//Portals/56/doc...
1,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2014-02629-CM,9/4/2023,10/3/2023,https://www.sam.usace.army.mil//Portals/56/doc...
2,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2019-01224-WPP,9/4/2023,9/25/2023,https://www.sam.usace.army.mil//Portals/56/doc...
3,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2007-00574-EG,8/28/2023,9/27/2023,https://www.sam.usace.army.mil//Portals/56/doc...
4,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-00799-CF,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...
...,...,...,...,...,...
2365,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2008-3113-CL,3/4/2013,4/2/2013,https://www.sam.usace.army.mil/http://www.mvn....
2366,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-MCM 2013,3/1/2013,3/31/2013,https://www.sam.usace.army.mil/http://www.mvn....
2367,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-1997-1616-EOO,2/25/2013,3/26/2013,https://www.sam.usace.army.mil/http://www.mvn....
2368,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2012-2234-CL,2/25/2013,3/24/2013,https://www.sam.usace.army.mil/http://www.mvn....


In [16]:
web_list = pd.read_csv("new_orleans_weblist.csv")

# Get a subset of the weblist
web_list_sub = web_list.loc[9:108].reset_index(drop = True)

## Step2: Scrape the webpage

In [6]:
# Scrape the webpage for each public notice on the weblist 
web_add_df = pd.DataFrame([scrape_webpage.web_extraction(x, 0) for x in web_list_sub["web_link"]],
                            columns = ['web_applicant_contents', 
                                       'web_location', 
                                       'web_character', 
                                       'web_mitigation'])
# Merge with weblist table
web_df = web_list_sub.join(web_add_df)

web_df.head()

,Unnamed: 0,web_link,web_title,published_date,web_expire_date,pdf_url,web_applicant_contents,web_location,web_character,web_mitigation
0,9,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2007-00574-EG,8/28/2023,9/27/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Cooper Consolidated, c/o Gulf South Engineerin...","At approximately Latitude: 30.046667, Longitud...",Expand an existing barge fleet to add one (1) ...,None
1,10,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-00799-CF,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Scott Thomas Development, LLC c/o: Trusted Com...","Located in Brusly, Louisiana, within West Bato...",The applicant has requested Department of the ...,The applicant proposes to avoid direct impacts...
2,11,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2023-00140-ECS,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"St. Charles Parish Government, c/o: Civix, Ms....","Located in Section 20, Township 15 South, Rang...",The proposed project is to widen the existing ...,None
3,12,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-00476-ECS,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"STOA Group, c/o: Hydrik Wetlands Consultants, ...","Located northerly of McKinney Road, Slidell, i...","The proposed project includes the clearing, gr...",None
4,13,http://www.mvn.usace.army.mil/Missions/Regulat...,MVN-2022-01256-WPP,8/21/2023,9/11/2023,https://www.sam.usace.army.mil//Portals/56/doc...,"Morganfield South, LLC, c/o: Rhett Francois, 1...","Located in Calcasieu Parish, near the communit...","Clear, grade, excavate and fill (approximately...",None


## Step3: Scrap PDFs

#### TEST

In [3]:
pdf_url = "https://www.sam.usace.army.mil/Portals/56/docs/regulatory/publicnotices/2022_01085_PNall_09252023.pdf?ver=UA0fmhXwaM-p7QhwJA_IIw%3d%3d"
# pdf_bytes = requests.get(pdf_url).content
# pdf_reader = pdf.PdfReader(io.BytesIO(pdf_bytes))
# p = 2
# pdf_p = pdf_reader.pages[p].extract_text()
# pdf_p

pdf_text = scrape_pdf.pdf_read(pdf_url, "new_orleans")
pdf_text
# scrape_pdf.get_comment_window(pdf_text, "new_orleans")
# scrape_pdf.get_pdf_applicant(pdf_text, "new_orleans")
# scrape_pdf.get_pdf_app_num_dist(pdf_text, "new_orleans")
pdf_character = scrape_pdf.get_pdf_character(pdf_text, "new_orleans")
pdf_character
scrape_pdf.get_pdf_acreage(pdf_character)
# char_sent_list = re.findall(r'.*?\D\.', pdf_character)
# char_sent_list
# for i in range(0, len(char_sent_list)):
#     if  any(w in char_sent_list[i] for w in ["impact", "affect", "loss"]):
#         acre_list = re.findall(r'([^,]*?\d*,?\d*\s?\.?\s?\d*-?\s?acres?.+?\D)(,|\.|\sand)', char_sent_list[i])
# acre_list
# re.search(r'[^A-Za-z]+(?=o?f?\s?a?n?\s?acres?\)?)', acre_list[0][0]).group()
# re.search(r'' + acre + '.+(of|to)(.*?(habitat|wetland|pond|water\s?bottom|marsh|water))',
#                                       acre_list[0][0]).group(2).strip()
# re.search(r'(permanent[a-z]*|temporar[a-z]*)', acre_list[0][0]).group().strip()

[('9.45', 'water botm', None), ('0.68', 'wetland', 'permanently')]

In [3]:
pdf_df = pd.DataFrame([scrape_pdf.pdf_extraction(x, "new_orleans") for x in web_df["pdf_url"]],
                            columns = ['pdf_special',
                                       'pdf_comment_window',
                                       'pdf_permit_app_num',
                                       'pdf_dist_code',
                                       'pdf_dist_name',  
                                       'pdf_manager_name', 
                                       'pdf_manager_phone', 
                                       'pdf_manager_email',
                                       'pdf_applicant_contents',
                                       'pdf_applicant',
                                       'pdf_contract',
                                       'pdf_location', 
                                       'pdf_character', 
                                       'pdf_mitigation',
                                       'pdf_county',
                                       'pdf_parish',
                                       'pdf_city',
                                       'pdf_hydrologic_unit_code', 
                                       'pdf_longitude',
                                       'pdf_latitude',
                                       'pdf_wqc',
                                       'pdf_cup',
                                       'pdf_acre1',
                                       'pdf_acre_type1', 
                                       'pdf_acre_term1',
                                       'pdf_acre2',
                                       'pdf_acre_type2', 
                                       'pdf_acre_term2', 
                                       'pdf_acre3',
                                       'pdf_acre_type3', 
                                       'pdf_acre_term3',
                                       'pdf_acre4',
                                       'pdf_acre_type4', 
                                       'pdf_acre_term4',
                                       'pdf_acre5',
                                       'pdf_acre_type5', 
                                       'pdf_acre_term5', 
                                       'pdf_acre6',
                                       'pdf_acre_type6', 
                                       'pdf_acre_term6', 
                                       'pdf_lf1', 
                                       'pdf_lf_type1', 
                                       'pdf_ldf_term1', 
                                       'pdf_lf2', 
                                       'pdf_lf_type2', 
                                       'pdf_ldf_term2', 
                                       'pdf_lf3', 
                                       'pdf_lf_type3', 
                                       'pdf_ldf_term3', 
                                       'pdf_full_text'])

# # Merge with RSS feed table
final_df = web_df.join(pdf_df)

# Add a new column to track the number of errors for each webpage
final_df["error"] = final_df.apply(lambda row: row.str.contains("ERROR").sum(), axis = 1)

final_df

UnboundLocalError: cannot access local variable 'special' where it is not associated with a value

In [6]:
final_df.to_csv("new_orleans_subset.csv")